# Run circuits in device emulators

# Implement Noise Model

In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
from qiskit.tools.jupyter import *
import warnings
# warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals

In [2]:
from qiskit.providers.fake_provider import *

In [9]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
shot = int(np.floor(1800000 / 631))

In [10]:
shot

2852

## Hamiltonian conversion

By using SHVQE, we need to apply the inversal of Heisenberg part to the original Hamiltonian. First, load the original Hamiltonian from `Hamiltonian/OHhamiltonian.txt`.

In [11]:
with open('Hamiltonian/OHhamiltonian.txt', 'r') as file:
    pauli_text_lines = file.readlines()
paulis = []
weight = []
for line in pauli_text_lines:
    line = line.replace(' ', '')
    coeff, pauli_text_string = line.split("*")
    coeff = float(coeff)
    weight.append(coeff)
    # pauli_text_string = pauli_text_string[::-1] # TODO: this is a quick but wrong fix for qubits order
    pauli_text_string = pauli_text_string.replace('\n', '')
    paulis.append(pauli_text_string)

Then, we use `Stim` to apply the inversal of Heisenberg part to Hamiltonian. In our protocal, the Heisenberg part consists of `cz` gates, which commute with each other. The inversal of the Heisenberg part is itself.

In [12]:
import stim

Load Heisenberg circuit.

In [13]:
heisenberg_text_combined = ''

with open('saved_models/shvqe_clifford_ncz10_hei.qasm', 'r') as file:
    heisenberg_text_lines = file.readlines()
    heisenberg_text_combined = ''.join(heisenberg_text_lines[3:]) # omit the header

In [14]:
import re


# Define the regular expression pattern
pattern = r"q\[(\d+)\],q\[(\d+)\]"

# Define the replacement string
replacement = r"\1 \2"

# Use the sub() function to replace the matches
stim_text = re.sub(pattern, replacement, heisenberg_text_combined).replace(';', '')

Apply the Heisenberg circuit to each Pauli string.

In [15]:
%%time

heisenberg_circ = stim.Circuit(stim_text)
paulis_new = []

for p in paulis:
    ps  = stim.PauliString(p.replace('I', '_'))
    ps = ps.after(heisenberg_circ)
    paulis_new.append(ps.__str__().replace('_', 'I'))

CPU times: user 20.3 ms, sys: 540 µs, total: 20.8 ms
Wall time: 20.4 ms


Construct the new Hamiltonian as observable. The number of Pauli strings is not changed by Clifford.

In [16]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp(paulis_new, weight)
print(f">>> Observable size: {observable.size}")

>>> Observable size: 631


# Circuit Transpile and Execution

In [17]:
from qiskit import qasm2
from qiskit_aer.primitives import Estimator

circuit = qasm2.load('saved_models/shvqe_clifford_ncz10_sch.qasm')

Reverse qubits to respect qiskit convention.

In [18]:
circuit = circuit.reverse_bits()

In [19]:
# circuit.draw('mpl');

Transpile the circuit based on given system model from IBMQ_Montreal. **We use default transpiler from qiskit.**

In [20]:
system_model = FakeMontreal()

In [21]:
# transpiled_circuit = transpile(circuit, backend=system_model)

In [22]:
# transpiled_circuit.draw();

## Noiseless
Evaluate on the qiskit Estimator. Consider noiseless first.

In [23]:
estimator_noiseless = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': None # noiseless
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [24]:
%%time
job_noiseless = estimator_noiseless.run(circuit, observable)
result_noiseless = job_noiseless.result()
# print(f">>> {result}")
result_noiseless.values

CPU times: user 1d 1h 15min 23s, sys: 3min 46s, total: 1d 1h 19min 9s
Wall time: 20min 35s


array([-78.75835435])

In [25]:
def get_error_rate(val):
    ref_value = -74.38714627
    nuclear_repulsion_energy = 4.365374966545
    energy_with_nuclear_repulsion = val + nuclear_repulsion_energy
    error_rate = abs(abs(ref_value - energy_with_nuclear_repulsion) / ref_value * 100)
    return error_rate

In [26]:
get_error_rate(result_noiseless.values)

array([0.00784156])

## Noise models (TODO)
### Fake Kolkata

In [27]:
import pickle

In [28]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model_fakekolkata = noise.NoiseModel.from_dict(pickle.load(file))

/tmp/ipykernel_129130/3864961774.py:2: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.10.0 and will be removed no earlier than 3 months from that release date.
  noise_model_fakekolkata = noise.NoiseModel.from_dict(pickle.load(file))


In [29]:
noise_model_fakekolkata

<NoiseModel on ['', 'sx', 'x', 'measure', 'id', 'reset', 'cx']>

In [30]:
estimator_fakekolkata = Estimator(
    backend_options = {
        # simulation options
        'method': 'statevector',
        'device': 'CPU',
        'max_parallel_threads' : 0,
        'max_parallel_shots' : 0,
        'statevector_parallel_threshold' : 5,
        'coupling_map' : system_model.configuration().coupling_map,
        'noise_model': noise_model_fakekolkata # noiseless
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler' : seed_transpiler,
    },
    skip_transpilation=False
)

In [31]:
%%time

job_fakekolkata = estimator_fakekolkata.run(circuit, observable)
result_fakekolkata = job_fakekolkata.result()
result_fakekolkata.values

CPU times: user 6h 54min 24s, sys: 4min 54s, total: 6h 59min 19s
Wall time: 5min 31s


array([-76.66606724])

In [32]:
get_error_rate(result_fakekolkata.values)

array([2.80485823])